<a href="https://colab.research.google.com/github/ammtjm/Stock_Data_Analysis/blob/main/3%E6%88%A6%E7%95%A5%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dash

In [9]:
import pandas as pd

# Load the CSV file
file_path = 'sp500指数ヒストリカルデータ.csv'
sp500_data = pd.read_csv(file_path)

# Display the first few rows of the data
sp500_data.head()


,Date,Open,High,Low,Close,Adj Close,Volume
0,2008-09-29,1209.069946,1209.069946,1106.420044,1106.420044,1106.420044,7305060000
1,2008-09-30,1113.780029,1168.030029,1113.780029,1166.359985,1166.359985,4937680000
2,2008-10-01,1164.170044,1167.030029,1140.770020,1161.060059,1161.060059,5782130000
3,2008-10-02,1160.640015,1160.640015,1111.430054,1114.280029,1114.280029,6285640000
4,2008-10-03,1115.160034,1153.819946,1098.140015,1099.229980,1099.229980,6716120000


In [10]:
import numpy as np

# Define the RSI strategy with history
def compute_RSI(data, window):
    diff = data.diff(1).dropna()
    gain = 0 * diff
    loss = 0 * diff
    gain[diff > 0] = diff[diff > 0]
    loss[diff < 0] = abs(diff[diff < 0])
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def rsi_strategy_with_history(data):
    cash = 1.0
    position = 0
    entry_price = 0
    in_trade = False
    history = [cash]

    for i in range(len(data)):
        if not in_trade and data['rsi'].iloc[i] > 30:
            # Buy
            entry_price = data['close'].iloc[i]
            position = cash / entry_price
            cash = 0
            in_trade = True
        elif in_trade and data['rsi'].iloc[i] > 70:
            # Sell
            cash = position * data['close'].iloc[i]
            position = 0
            in_trade = False
        # Update history with current value
        current_value = cash + position * data['close'].iloc[i]
        history.append(current_value)

    return history

# Calculate RSI values and run the RSI strategy
sp500_data['close'] = sp500_data['Close']
sp500_data['rsi'] = compute_RSI(sp500_data['close'], 14)
rsi_history = rsi_strategy_with_history(sp500_data)

# Keep the last 1000 days of data for comparison
rsi_history = rsi_history[-1000:]
rsi_history[:10]  # Display the first 10 elements of the RSI history


[3.2818845242086727,
 3.3285427403766534,
 3.3136365451516,
 3.262073645973318,
 3.2917731711804423,
 3.312892185110075,
 3.349131838226428,
 3.3444861947128266,
 3.377782817798135,
 3.3710287964893997]

In [12]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
# Define the training period as up to 2019-10-08
train_data = sp500_data[sp500_data['Date'] < '2019-10-09']

# Get closing prices for training
closing_prices_train = train_data['Close'].values

# Create binary labels: 1 if the price goes up, 0 if it goes down
labels = (sp500_data['Close'].diff() > 0).astype(int).shift(-1).dropna()

# Define the training data and labels
X_train_class = np.array([closing_prices_train[i-20:i] for i in range(20, len(closing_prices_train))])
y_train_class = labels.iloc[:len(closing_prices_train) - 20]

# Train the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100,random_state = 0)
rf_classifier.fit(X_train_class, y_train_class)

# Initialize variables for trading simulation
capital = 1.0
position = 0.0
rf_history = [capital]

# Run the rolling prediction and trading simulation using the classifier
for i in range(len(sp500_data) - 20):
    # Get the past 20 days' prices
    X_test = sp500_data['Close'].values[i:i+20].reshape(1, -1)
    # Predict the next day's price movement
    prediction = rf_classifier.predict(X_test)

    # Trading logic: Go long if the predicted movement is 1, close position if it's 0
    current_price = sp500_data['Close'].iloc[i + 20]
    if prediction == 1 and capital > 0:  # Buy
        position += capital / current_price
        capital = 0.0
    elif prediction == 0 and position > 0:  # Sell
        capital += position * current_price
        position = 0.0

    # Update history with current value
    current_value = capital + position * current_price
    rf_history.append(current_value)

# Display the first 10 elements of the trading history using RandomForestClassifier
rf_history[:10]



[1.0,
 1.0,
 1.1078900587797356,
 1.1078900587797356,
 1.1078900587797356,
 1.1078900587797356,
 1.1078900587797356,
 1.1078900587797356,
 1.1078900587797356,
 1.1078900587797356]

In [13]:
def trading_simulation(prices, predictions):
    capital = 1.0
    position = 0.0
    history = [capital]

    for i in range(len(prices) - 1):
        if predictions[i] > prices[i] and capital > 0:  # Buy
            position += capital / prices[i + 1]
            capital = 0.0
        elif predictions[i] < prices[i] and position > 0:  # Sell
            capital += position * prices[i + 1]
            position = 0.0

        # Update history with current value
        current_value = capital + position * prices[i + 1]
        history.append(current_value)

    return history
# Define the Kalman Filter and SSM prediction functions
def kalman_filter_v9(y, Z, T, R, Q, a1, P1):
    n = len(y)
    alpha_predicted = np.zeros((n, 2, 1))
    P_predicted = np.zeros((n, 2, 2))
    alpha_filtered = np.zeros((n, 2, 1))
    P_filtered = np.zeros((n, 2, 2))
    alpha_smoothed = np.zeros((n, 2, 1))
    P_smoothed = np.zeros((n, 2, 2))
    P_pair = np.zeros((n, 2, 2))

    for t in range(n):
        if t == 0:
            alpha_predicted[t] = a1.reshape(2, 1)
            P_predicted[t] = P1
        else:
            alpha_predicted[t] = np.dot(T, alpha_filtered[t-1])
            P_predicted[t] = np.dot(T, np.dot(P_filtered[t-1], T.T)) + np.dot(R, np.dot(Q, R.T))

        v = y[t] - np.dot(Z, alpha_predicted[t])
        F = np.dot(Z, np.dot(P_predicted[t], Z.T))
        K = np.dot(np.dot(P_predicted[t], Z.T), np.linalg.inv(F))
        alpha_filtered[t] = alpha_predicted[t] + K * v
        P_filtered[t] = P_predicted[t] - np.dot(K, np.dot(Z, P_predicted[t]))

    alpha_smoothed[-1] = alpha_filtered[-1]
    P_smoothed[-1] = P_filtered[-1]
    for t in range(n-2, -1, -1):
        L = np.dot(P_filtered[t], np.dot(T.T, np.linalg.inv(P_predicted[t+1])))
        alpha_smoothed[t] = alpha_filtered[t] + np.dot(L, (alpha_smoothed[t+1] - np.dot(T, alpha_filtered[t])))
        P_smoothed[t] = P_filtered[t] + np.dot(np.dot(L, (P_smoothed[t+1] - P_predicted[t+1])), L.T)

    return alpha_predicted, P_predicted, alpha_filtered, P_filtered, alpha_smoothed, P_smoothed, P_pair

def ssm_one_day_prediction_v4(y, Z, T, R, Q, a1, P1):
    y_subset = y[-200:]  # Take the last 200 days
    prediction = kalman_filter_v9(y_subset, Z, T, R, Q, a1, P1)[-3][-1]
    return prediction[0, 0]

# Initialize parameters for State Space Model
closing_prices = sp500_data['close'].values
Z = np.array([[1.0, 0.0]])
T = np.array([[1, 1], [0, 1]])
R = np.eye(2)
Q = 1.0
a1 = np.array([closing_prices[0], 0.0])
P1 = np.eye(2)

# State Space Model Predictions for 1 day ahead for 1000 days using previous 200 days
ssm_one_day_predictions_rolling = [ssm_one_day_prediction_v4(closing_prices[i-1200:i], Z, T, R, Q, a1, P1) for i in range(-1000, 0)]
ssm_history = trading_simulation(closing_prices[-1000:], ssm_one_day_predictions_rolling)

ssm_history[:10]  # Display the first 10 elements of the SSM history


[1.0,
 1.0,
 1.0,
 1.0,
 1.0091044925499415,
 1.0155785995817375,
 1.0155785995817375,
 1.0155785995817375,
 1.0155785995817375,
 1.0135479067064799]

In [14]:
# Define function to calculate Maximum Drawdown
def calculate_max_drawdown(history):
    max_drawdown = 0
    peak = history[0]
    for value in history:
        if value > peak:
            peak = value
        drawdown = (peak - value) / peak
        if drawdown > max_drawdown:
            max_drawdown = drawdown
    return max_drawdown

# Define function to calculate Drawdown Duration
def calculate_drawdown_duration(history):
    max_duration = 0
    duration = 0
    peak = history[0]
    for value in history:
        if value >= peak:
            peak = value
            duration = 0
        else:
            duration += 1
            if duration > max_duration:
                max_duration = duration
    return max_duration

# Define function to calculate Sharpe Ratio (assuming risk-free rate is 0)
def calculate_sharpe_ratio(history):
    returns = np.diff(history) / history[:-1]
    sharpe_ratio = np.mean(returns) / np.std(returns)
    return sharpe_ratio

# Define function to calculate Final Profit
def calculate_final_profit(history):
    return history[-1] - history[0]

# Calculate metrics for each strategy
strategies = {
    'RSI': rsi_history,
    'Random Forest': rf_history,
    'SSM': ssm_history
}

metrics = {}
for strategy, history in strategies.items():
    max_drawdown = calculate_max_drawdown(history)
    drawdown_duration = calculate_drawdown_duration(history)
    sharpe_ratio = calculate_sharpe_ratio(history)
    final_profit = calculate_final_profit(history)

    metrics[strategy] = {
        'Max Drawdown': max_drawdown,
        'Drawdown Duration': drawdown_duration,
        'Sharpe Ratio': sharpe_ratio,
        'Final Profit': final_profit
    }

metrics_df = pd.DataFrame(metrics).T
metrics_df


,Max Drawdown,Drawdown Duration,Sharpe Ratio,Final Profit
RSI,0.316338,432.0,0.042642,2.131834
Random Forest,0.254251,468.0,0.033820,1.857273
SSM,0.306696,758.0,0.019112,0.161166


In [15]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
import dash_table
from sklearn.ensemble import RandomForestRegressor

# Load SP500 data
# データの読み込みと前処理
metrics_df = metrics_df  # 上記で計算した metrics_df
rsi_history = rsi_history  # 上記で計算した rsi_history
rf_history = rf_history  # 上記で計算した rf_history
ssm_history = ssm_history  # 上記で計算した ssm_history
sp500_data = sp500_data  # SP500のデータ

# Find the position of the reference date within the data
reference_date = '2019-10-08'
reference_index = sp500_data[sp500_data['Date'] == reference_date].index[0]

# Calculate the relative reference index within the last 1000 days
relative_reference_index = reference_index - (len(sp500_data) - 1000)

# Cut the rf_history to include only data from '2019-10-08' onwards
rf_history = rf_history[reference_index:]

# Reset the rf_history based on the first value
rf_history = [value / rf_history[0] for value in rf_history]

# Check if the reference index is within the range of the last 1000 days
if relative_reference_index >= 0:
    # Define a function to reset cumulative profit based on the reference index
    def reset_cumulative_profit(history, relative_reference_index):
        reference_value = history[relative_reference_index]
        adjusted_history = [value / reference_value for value in history]
        return adjusted_history

    # Adjust the cumulative profit histories from the reference index
    adjusted_rsi_history = reset_cumulative_profit(rsi_history, relative_reference_index)
    adjusted_rf_history = reset_cumulative_profit(rf_history, relative_reference_index)
    adjusted_ssm_history = reset_cumulative_profit(ssm_history, relative_reference_index)



# Calculate the metrics again based on the adjusted histories
adjusted_metrics = {
    'RSI': {
        'Max Drawdown': calculate_max_drawdown(adjusted_rsi_history),
        'Drawdown Duration': calculate_drawdown_duration(adjusted_rsi_history),
        'Sharpe Ratio': calculate_sharpe_ratio(adjusted_rsi_history),
        'Final Profit': calculate_final_profit(adjusted_rsi_history),
    },
    'Random Forest': {
        'Max Drawdown': calculate_max_drawdown(adjusted_rf_history),
        'Drawdown Duration': calculate_drawdown_duration(adjusted_rf_history),
        'Sharpe Ratio': calculate_sharpe_ratio(adjusted_rf_history),
        'Final Profit': calculate_final_profit(adjusted_rf_history),
    },
    'SSM': {
        'Max Drawdown': calculate_max_drawdown(adjusted_ssm_history),
        'Drawdown Duration': calculate_drawdown_duration(adjusted_ssm_history),
        'Sharpe Ratio': calculate_sharpe_ratio(adjusted_ssm_history),
        'Final Profit': calculate_final_profit(adjusted_ssm_history),
    }
}

# 1. ホールド戦略の累積利益計算
hold_prices = sp500_data['Close'].values[reference_index:]
hold_history = (hold_prices / hold_prices[0]).tolist()
# 2. 指標の計算
adjusted_metrics['Hold'] = {
    'Max Drawdown': calculate_max_drawdown(hold_history),
    'Drawdown Duration': calculate_drawdown_duration(hold_history),
    'Sharpe Ratio': calculate_sharpe_ratio(hold_history),
    'Final Profit': calculate_final_profit(hold_history),
}

# Create a DataFrame for the adjusted metrics
adjusted_metrics_df = pd.DataFrame(adjusted_metrics).T

# インデックスをリセットして新しいStrategy列を作成
adjusted_metrics_df = adjusted_metrics_df.reset_index().rename(columns={'index': 'Strategy'})

# Initialize the Dash application
app = dash.Dash(__name__)

# Define the layout of the application
app.layout = html.Div([
    dcc.Tabs(id="tabs", value='tab1', children=[
        dcc.Tab(label='Metrics', value='tab1'),
        dcc.Tab(label='Profit Graph', value='tab2'),
    ]),
    html.Div(id='tabs-content')
])

# Define the callback to update the content based on the selected tab
@app.callback(dash.dependencies.Output('tabs-content', 'children'),
              [dash.dependencies.Input('tabs', 'value')])
def update_tab(tab_name):
    if tab_name == 'tab1':
        return html.Div([
            html.H4('Trading Strategy Metrics'),
            dash_table.DataTable(
                id='table',
                columns=[{"name": i, "id": i} for i in adjusted_metrics_df.columns],
                data=adjusted_metrics_df.to_dict('records'),
            )
        ])
    elif tab_name == 'tab2':
        trace_rsi = go.Scatter(
            x=sp500_data['Date'].tail(len(adjusted_rsi_history)),
            y=adjusted_rsi_history,
            mode='lines',
            name='RSI History'
        )
        trace_rf = go.Scatter(
            x=sp500_data['Date'].tail(len(adjusted_rf_history)),
            y=adjusted_rf_history,
            mode='lines',
            name='RF History'
        )
        trace_ssm = go.Scatter(
            x=sp500_data['Date'].tail(len(adjusted_ssm_history)),
            y=adjusted_ssm_history,
            mode='lines',
            name='SSM History'
        )
        # 3. プロットの追加
        trace_hold = go.Scatter(
            x=sp500_data['Date'].tail(len(hold_history)),
            y=hold_history,
            mode='lines',
            name='Hold History'
        )
        layout = go.Layout(
            title='Adjusted Profit Histories for RSI, RF, SSM, and Hold Strategies',
            xaxis=dict(title='Date'),
            yaxis=dict(title='Profit')
        )
        return dcc.Graph(
            id='graph',
            figure={
                'data': [trace_rsi, trace_rf, trace_ssm, trace_hold],
                'layout': layout
            }
        )

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>